# Segment Anything Model for Geospatial Data 

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/satellite.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

This notebook shows how to use segment satellite imagery using the Segment Anything Model (SAM) with a few lines of code. 

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

this is `大手町` Area version custom.

Inference takes about 30 minutes.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.


In [50]:
%pip install -q segment-geospatial leafmap localtileserver

## Import libraries

In [51]:
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff

## Create an interactive map

In [52]:
m = leafmap.Map(center=[35.685518, 139.764275], zoom=19)
m.add_basemap('SATELLITE')
m

Map(center=[35.685518, 139.764275], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [53]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

In [54]:
bbox = [139.76, 35.68, 139.77, 35.69] # 35.685518, 139.764275

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [55]:
image = 'satellite.tif'
# image = '/path/to/your/own/image.tif'

In [56]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite', overwrite=True)

Downloaded image 1/1110
Downloaded image 2/1110
Downloaded image 3/1110
Downloaded image 4/1110
Downloaded image 5/1110
Downloaded image 6/1110
Downloaded image 7/1110
Downloaded image 8/1110
Downloaded image 9/1110
Downloaded image 10/1110
Downloaded image 11/1110
Downloaded image 12/1110
Downloaded image 13/1110
Downloaded image 14/1110
Downloaded image 15/1110
Downloaded image 16/1110
Downloaded image 17/1110
Downloaded image 18/1110
Downloaded image 19/1110
Downloaded image 20/1110
Downloaded image 21/1110
Downloaded image 22/1110
Downloaded image 23/1110
Downloaded image 24/1110
Downloaded image 25/1110
Downloaded image 26/1110
Downloaded image 27/1110
Downloaded image 28/1110
Downloaded image 29/1110
Downloaded image 30/1110
Downloaded image 31/1110
Downloaded image 32/1110
Downloaded image 33/1110
Downloaded image 34/1110
Downloaded image 35/1110
Downloaded image 36/1110
Downloaded image 37/1110
Downloaded image 38/1110
Downloaded image 39/1110
Downloaded image 40/1110
Downloade

In [57]:
m.add_raster(image, layer_name='Image')
m

Map(center=[35.685518, 139.764275], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Initialize SAM class

In [58]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [59]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

## Segment the image

In [60]:
mask = 'segment.tiff'
sam.generate(image, mask)

100%|██████████| 285/285 [33:38<00:00,  7.08s/it]


## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [61]:
vector = 'segment.gpkg'
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [62]:
shapefile = 'segment.shp'
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [ ]:
# style = {
#     'color': '#9900ff',
#     'weight': 2,
#     'fillColor': '#7c4199',
#     'fillOpacity': 0.35,
# }
# m.add_vector(vector, layer_name='Vector', style=style)
# m